## Chapter 4. Molecular Hamiltonians

#### In this section, we are going to discuss about applying VQE to find the lowest energy on a given complicating moleculars.

#### This chapter contains several topics

- Born-Oppenheimer approximation
- Fock space
- Fermionic creation and annihilation operators
- Molecular Hamiltonian in the Hatree-Fock orbitals basis
- Basis sets
- Constructing a fermionic Hamiltonian with Qiskit Nature
- Fermion to qubit mappings
- Constructing a qubit Hamiltonian operator with Qiskit Nature

### I. Installation

#### This part informs several python packages required for our tutorial.

In [1]:
!pip install numpy      # an open source Python library that is used in almost every field of science and engineering
!pip install qiskit     # an open source SDK for working with quantum computers at the level of pulses, circuits, and application modules
!pip install 'qiskit[visualization]'    # this is for qiskit visualization support
!pip install qiskit.utils               # updated qiskit
!pip install qiskit-nature              # a unique platform that bridges the gap between natural sciences and quantum simulations
!pip install pyscf                      # Python-based Simulations of Chemistry Framework, an open source collection of electronic structure modules powered by Python

DEPRECATION: pandas 0.24.2 has a non-standard dependency specifier pytz>=2011k. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pandas or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of qiskit to determine which version is compatible with other requirements. This could take a while.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for qiskit: filename=qiskit-0.43.3-py3-none-any.whl size=7632 sha256=9400e6d07e85274171cb60dda32f385329c8648e1f97169d2e1906c

In [2]:
#import packages
import numpy as np

#visualization tools
import matplotlib.pyplot as plt

#qiskit environments and Latex representations of complex array
from qiskit.visualization import array_to_latex, plot_bloch_vector, plot_bloch_multivector, plot_state_qsphere, plot_state_city
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, transpile
from qiskit import execute, Aer
import qiskit.quantum_info as qi
from qiskit.quantum_info import SparsePauliOp   #updated qiskit
from qiskit.extensions import Initialize
#from qiskit.providers.aer import extensions     #import aer snapshot instructions

#qiskit nature libraries
from qiskit import Aer
#from qiskit_nature.drivers import UnitsType, Molecule

#from qiskit_nature.drivers.second_quantization import ElectronicStructureDriverType, ElectronicStructureMoleculeDriver
#from qiskit_nature.problems.second_quantization import ElectronicStructureProblem
#from qiskit_nature.mappers.second_quantization import ParityMapper, JordanWignerMapper, BravyiKitaevMapper
#from qiskit_nature.converters.second_quantization import QubitConverter

# Electronic Structure Problems with v0.5
# https://qiskit.org/documentation/nature/migration/00b_Electronic_structure_with_v0.5.html
from qiskit_nature.second_q.formats.molecule_info import MoleculeInfo
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import QubitConverter
from qiskit_nature.second_q.mappers import ParityMapper, JordanWignerMapper, BravyiKitaevMapper
from qiskit_nature.second_q.properties import ParticleNumber
from qiskit_nature.second_q.transformers import ActiveSpaceTransformer, FreezeCoreTransformer
from qiskit_nature.second_q.operators import FermionicOp
# https://qiskit.org/ecosystem/nature/migration/0.6_c_qubit_converter.html#paulisumop-vs-sparsepauliop
from qiskit_nature import settings

#from qiskit_nature.operators.second_quantization import FermionicOp
#from qiskit_nature.circuit.library.initial_states import HartreeFock
#from qiskit_nature.circuit.library.ansatzes import UCCSD

#Qiskit Nature property framework
#from qiskit_nature.properties import Property, GroupedProperty
# https://qiskit.org/documentation/nature/migration/00b_Electronic_structure_with_v0.5.html
from qiskit_nature.second_q.operators import ElectronicIntegrals

#math libraries
import cmath
import math

### II. Born-Oppenheimer Approximation

#### Since the atomic orbital of an elebron in an atom and the molecular orbital of an electron in a molecule are time-independent stationary states, they can be solved by using time-independent Schrödinger equation: $$ \hat{H} |\Psi \rangle = E | \Psi \rangle $$

#### Where $\hat{H}$ is the non-relateve Hamiltonian operator obtained by quantizing the classical energy in Hamiltonian form (first quantization), and it represents the total energy($E$) of all its particles; $N$ electrons and $M$ nuclei.

#### Let's define a system containing molecules on the coordinates. The origin of the coordinate system can be placed anywhere in free space. It is shown in the figure below

![Coordinate System with Molecules](./img/coordinates.png)

#### The Molecular Hamiltonian in the coordinates is: $$ \hat{H} = -\sum_I \frac{\nabla_{r_i}^{2}}{M_I} -\sum_i \frac{\nabla_{r_i}^{2}}{m_e} -\sum_{I} \sum_{i} \frac{Z_I e^2}{|R_I - r_i|} + \sum_i \sum_{j > i} \frac{e^2}{|r_i }